<a href="https://colab.research.google.com/github/fsternini/brats-task2/blob/development/brats_task2_launcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# First, verification of Python version is completed
!which python # should return /usr/local/bin/python
!python --version

/usr/local/bin/python
Python 3.10.6


In [2]:
# Google Drive is mounted, to reach all relevant data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Change the working directory reach the local repo

%cd /content/drive/MyDrive/Data Science/Github/brats-task2

/content/drive/MyDrive/Data Science/Github/brats-task2


In [4]:
#!git clone https://github.com/fsternini/brats-task2.git

In [8]:
# Switch to the most updated branch
# Update the branch to the relevant one
!git switch development

Already on 'development'
Your branch is up to date with 'origin/development'.


In [6]:
# Collect the most updated version of the repo
!git pull

Already up to date.


In [7]:
# Install all requirements for the code
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.3/766.3 kB 8.6 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Data Science/Github/brats-task2/code')
import common.common

In [ ]:
from common.common import DatasetCreator

In [ ]:
dataset_creator = DatasetCreator('/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset')